# This model is implemented without release!

In [1]:
using JuMP, Gurobi
include("scenarios_instance_reader.jl")
include("ship_reader.jl")

data_file_F = "S_7_1_5_False.txt"
data_file_CP = "S_7_1_5_False_CP_LD.txt"
data_file_ship = "Medium_ship.txt"

data_F = read_scenario_instance(data_file_F)
data_CP = read_scenario_instance(data_file_CP)
data_ship = read_ship_instance(data_file_ship)

ShipInstance(24, 126, 20, [1.0, 2.0, 4.0, 5.0, 7.0, 9.0, 11.0, 13.0, 15.0, 17.0  …  110.0, 112.0, 114.0, 116.0, 118.0, 120.0, 122.0, 124.0, 125.0, 126.0], [0.0, 3.0, -1.0, 0.0, 6.0, -1.0, 8.0, -1.0, 10.0, -1.0  …  -1.0, 119.0, -1.0, 121.0, -1.0, 123.0, -1.0, 0.0, 0.0, 0.0], [Float64[], [1.0, 2.0, 4.0], [5.0, 7.0, 9.0], [11.0, 13.0, 15.0], [17.0, 19.0, 21.0], [23.0, 25.0, 27.0], [29.0, 31.0, 33.0], [35.0, 37.0, 39.0], [41.0, 43.0, 45.0], [47.0, 49.0, 51.0]  …  [77.0, 79.0, 81.0], [83.0, 85.0, 87.0], [89.0, 91.0, 93.0], Float64[], [95.0, 97.0, 98.0], [100.0, 102.0, 104.0], [106.0, 108.0, 110.0], [112.0, 114.0, 116.0], [118.0, 120.0, 122.0], [124.0, 125.0, 126.0]], [2, 2, 2, 2, 3, 3, 3, 3, 3, 3  …  22, 23, 23, 23, 23, 23, 23, 24, 24, 24], [10, 3, 108, 10, 20, 29, 3, 140, 3, 62  …  184, 13, 133, 4, 150, 4, 169, 93, 66, 93], [14, 7, 44, 14, 21, 21, 7, 57, 7, 35  …  69, 18, 55, 9, 61, 9, 64, 54, 45, 54], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [282.24, 141.12, 1025.2

In [2]:
# Count amount of 20' and 40' type containers
T_20 = count(x -> x.length == 20, data_CP.container_types)
T_40 = count(x -> x.length == 40, data_CP.container_types)

L = data_ship.n_locations          # Number of locations/blocks
P = data_CP.n_ports              # Number of ports to visit
pc = 1
N = data_F.n_scenarios

LD = data_CP.containers          # The number of container in each leg per container type
omega = data_F.containers            # The number of container in each leg per container type

transport_keys = collect(keys(LD))

TR = length(LD)  # Number of transports (sets of loading and discharge ports) #((P-1)*P/2)
TR_ON = Dict{Int, Vector{Int}}()    # Set of transports on the vessel at departure from port p
for p in 1:P
    TR_ON[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if load <= p < discharge]
end 
TR_A = Dict{Int, Vector{Int}}()     # Set of transports that is loading or unloading in port p
for p in 1:P
    TR_A[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if load == p || discharge == p]
end
TR_OV = Dict{Int, Vector{Int}}()    # Set of transports thet overstow containers to load or unload in port p
for p in 1:P
    TR_OV[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if load < p < discharge]
end
TR_DEP = Dict{Int, Vector{Int}}()   # TR_DEP: Set of transports departing from port p
for p in 1:P
    TR_DEP[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if load == p]
end
TR_FU = Dict{Int, Vector{Int}}()    # TR_FU: Set of transports departing from all later ports than port p
for p in 1:P
    TR_FU[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if load > p]
end


# Bruges ikke i denne model:
TR_DIS = Dict{Int, Vector{Int}}()   # Set of transporters that discharges at port p
for p in 1:P
    TR_DIS[p] = [t_idx for (t_idx, (load, discharge)) in enumerate(keys(LD)) if discharge == p]
end



C_20 = data_ship.location_TEU_capacity   # TEU capacity of each location
C_40 = data_ship.location_FEU_capacity   # FEU capacity of each location
C_R = data_ship.location_reefer_capacity # Number of reefer plugs in each location 

F_20 = C_R ./ C_20  # Factor for all non-reefer 20' containers
F_40 = C_R ./ C_40  # Factor for all non-reefer 40' containers

reefers20 = findall(x -> x.is_reefer == true && x.length == 20, data_CP.container_types)           # List of index for reefercontainers in C_20
reefers40 = findall(x -> x.is_reefer == true && x.length == 40, data_CP.container_types) .- T_40   # List of index for reefercontainers in C_40
         
L_O = Int.(data_ship.locations_over)    # List of on-deck locations
L_U = Int.(data_ship.locations_under)   # List of locations under a given location (0 if the given location is under-deck)

M = maximum(C_20)+maximum(C_40)

FactorK_O = 10

10

In [3]:
StochasticModel_Simple = Model(Gurobi.Optimizer)

Set parameter Username
Set parameter LicenseID to value 2709821
Academic license - for non-commercial use only - expires 2026-09-17


A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [ ]:
# Amount of containers in each location for each transport for the current port
@variable(StochasticModel_Simple, x_20[1:T_20, 1:TR, 1:L] >= 0)
@variable(StochasticModel_Simple, x_40[1:T_40, 1:TR, 1:L] >= 0)

# Continers not loaded from current port
@variable(StochasticModel_Simple, z_20[1:T_20, 1:TR] >= 0)
@variable(StochasticModel_Simple, z_40[1:T_40, 1:TR] >= 0)

### Stocastic variables:
# Amount of containers in each location for each transport for future ports
@variable(StochasticModel_Simple, s_20[1:N, 1:T_20, 1:TR, 1:L] >= 0)
@variable(StochasticModel_Simple, s_40[1:N, 1:T_40, 1:TR, 1:L] >= 0)

# Continers not loaded from future ports
@variable(StochasticModel_Simple, q_20[1:N, 1:T_20, 1:TR] >= 0)
@variable(StochasticModel_Simple, q_40[1:N, 1:T_40, 1:TR] >= 0)

# presence of 20' containers to load or unload? {no,yes}
@variable(StochasticModel_Simple, delta[1:N, 1:P, 1:L], Bin)

# The number of hatch overstow containers
@variable(StochasticModel_Simple, y_O[1:N, 1:P, 1:L] >= 0)

5×7×126 Array{VariableRef, 3}:
[:, :, 1] =
 y_O[1,1,1]  y_O[1,2,1]  y_O[1,3,1]  …  y_O[1,5,1]  y_O[1,6,1]  y_O[1,7,1]
 y_O[2,1,1]  y_O[2,2,1]  y_O[2,3,1]     y_O[2,5,1]  y_O[2,6,1]  y_O[2,7,1]
 y_O[3,1,1]  y_O[3,2,1]  y_O[3,3,1]     y_O[3,5,1]  y_O[3,6,1]  y_O[3,7,1]
 y_O[4,1,1]  y_O[4,2,1]  y_O[4,3,1]     y_O[4,5,1]  y_O[4,6,1]  y_O[4,7,1]
 y_O[5,1,1]  y_O[5,2,1]  y_O[5,3,1]     y_O[5,5,1]  y_O[5,6,1]  y_O[5,7,1]

[:, :, 2] =
 y_O[1,1,2]  y_O[1,2,2]  y_O[1,3,2]  …  y_O[1,5,2]  y_O[1,6,2]  y_O[1,7,2]
 y_O[2,1,2]  y_O[2,2,2]  y_O[2,3,2]     y_O[2,5,2]  y_O[2,6,2]  y_O[2,7,2]
 y_O[3,1,2]  y_O[3,2,2]  y_O[3,3,2]     y_O[3,5,2]  y_O[3,6,2]  y_O[3,7,2]
 y_O[4,1,2]  y_O[4,2,2]  y_O[4,3,2]     y_O[4,5,2]  y_O[4,6,2]  y_O[4,7,2]
 y_O[5,1,2]  y_O[5,2,2]  y_O[5,3,2]     y_O[5,5,2]  y_O[5,6,2]  y_O[5,7,2]

[:, :, 3] =
 y_O[1,1,3]  y_O[1,2,3]  y_O[1,3,3]  …  y_O[1,5,3]  y_O[1,6,3]  y_O[1,7,3]
 y_O[2,1,3]  y_O[2,2,3]  y_O[2,3,3]     y_O[2,5,3]  y_O[2,6,3]  y_O[2,7,3]
 y_O[3,1,3]  y_O[3,2,3]  y_O[3,

In [5]:
@objective(StochasticModel_Simple, Min, 
    sum(z_20[tau_20, t] for tau_20 in 1:T_20, t in 1:TR) +
    sum(z_40[tau_40, t] for tau_40 in 1:T_40, t in 1:TR) +
    sum( (1/N) *
        sum(FactorK_O * y_O[i, p, l] for i in 1:N, p in 1:P, l in 1:L) +
        sum(q_20[i, tau_20, t] for i in 1:N, tau_20 in 1:T_20, t in TR_FU[pc]) + # TR_FU er ikke defineret endnu.
        sum(q_40[i, tau_40, t] for i in 1:N, tau_40 in 1:T_40, t in TR_FU[pc])
    ))

z_20[1,1] + z_20[1,2] + z_20[1,3] + z_20[1,4] + z_20[1,5] + z_20[1,6] + z_20[1,7] + z_20[1,8] + z_20[1,9] + z_20[1,10] + z_20[1,11] + z_20[1,12] + z_20[1,13] + z_20[1,14] + z_20[1,15] + z_20[1,16] + z_20[1,17] + z_20[1,18] + z_20[1,19] + z_20[1,20] + z_20[1,21] + z_20[2,1] + z_20[2,2] + z_20[2,3] + z_20[2,4] + z_20[2,5] + z_20[2,6] + z_20[2,7] + z_20[2,8] + z_20[2,9] + [[...7038 terms omitted...]] + q_40[5,15,1] + q_40[5,15,3] + q_40[5,15,4] + q_40[5,15,7] + q_40[5,15,8] + q_40[5,15,9] + q_40[5,15,10] + q_40[5,15,11] + q_40[5,15,12] + q_40[5,15,14] + q_40[5,15,16] + q_40[5,15,17] + q_40[5,15,18] + q_40[5,15,20] + q_40[5,15,21] + q_40[5,16,1] + q_40[5,16,3] + q_40[5,16,4] + q_40[5,16,7] + q_40[5,16,8] + q_40[5,16,9] + q_40[5,16,10] + q_40[5,16,11] + q_40[5,16,12] + q_40[5,16,14] + q_40[5,16,16] + q_40[5,16,17] + q_40[5,16,18] + q_40[5,16,20] + q_40[5,16,21]

In [6]:
###  First stage:
# Loadlist constraint (4)
@constraint(StochasticModel_Simple, [tau_20 = 1:T_20, t = 1:TR], 
    sum(x_20[tau_20, t, l] for l in 1:L) 
    <= LD[transport_keys[t]][tau_20]          # Relaxed to not enclude all containers
    )
@constraint(StochasticModel_Simple, [tau_40 = 1:T_40, t = 1:TR], 
    sum(x_40[tau_40, t, l] for l in 1:L) 
    <= LD[transport_keys[t]][T_20 + tau_40]   # Relaxed to not enclude all containers
    )

# Minimise unloaded containers constraint
@constraint(StochasticModel_Simple, [tau_20 = 1:T_20, t = 1:TR],
    LD[transport_keys[t]][tau_20] -
    sum(x_20[tau_20, t, l] for l in 1:L)
    == z_20[tau_20, t]
    )
@constraint(StochasticModel_Simple, [tau_40 = 1:T_40, t = 1:TR],
    LD[transport_keys[t]][T_20 + tau_40] -
    sum(x_40[tau_40, t, l] for l in 1:L)
    == z_40[tau_40, t]
    )

16×21 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 -x_40[1,1,1] - x_40[1,1,2] - x_40[1,1,3] - x_40[1,1,4] - x_40[1,1,5] - x_40[1,1,6] - x_40[1,1,7] - x_40[1,1,8] - x_40[1,1,9] - x_40[1,1,10] - x_40[1,1,11] - x_40[1,1,12] - x_40[1,1,13] - x_40[1,1,14] - x_40[1,1,15] - x_40[1,1,16] - x_40[1,1,17] - x_40[1,1,18] - x_40[1,1,19] - x_40[1,1,20] - x_40[1,1,21] - x_40[1,1,22] - x_40[1,1,23] - x_40[1,1,24] - x_40[1,1,25] - x_40[1,1,26] - x_40[1,1,27] - x_40[1,1,28] - x_40[1,1,29] - x_40[1,1,30] - [[...67 terms omitted...]] - x_40[1,1,98] - x_40[1,1,99] - x_40[1,1,100] - x_40[1,1,101] - x_40[1,1,102] - x_40[1,1,103] - x_40[1,1,104] - x_40[1,1,105] - x_40[1,1,106] - x_40[1,1,107] - x_40[1,1,108] - x_40[1,1,109] - x_40[1,1,110] - x_40[1,1,111] - x_40[1,1,112] - x_40[1,1,113] - x_40[1,1,114] - x_40[1,1,115] - x_40[1,1,116] - x_40[1,1,117] - x_40[1,1,118] - x_40[1,1,119] - x_40[1,1,120

In [8]:
### Second stage:
# Capacity constraint (1)
@constraint(StochasticModel_Simple, [p = 1:P, l = 1:L],
    sum(x_20[tau_20, t, l] for tau_20 in 1:T_20, t in TR_ON[p]) + 2 *
    sum(x_40[tau_40, t, l] for tau_40 in 1:T_40, t in TR_ON[p]) +
    sum(s_20[i, tau_20, t, l] for i in 1:N, tau_20 in 1:T_20, t in TR_ON[p]) + 2 *
    sum(s_40[i, tau_40, t, l] for i in 1:N, tau_40 in 1:T_40, t in TR_ON[p])
    <= C_20[l]
    )

# Reafer capcity constraint (2)
@constraint(StochasticModel_Simple, [p = 1:P, l = 1:L],
    sum(x_20[tau_20, t, l] for tau_20 in 1:T_20, t in TR_ON[p]) + 2 *
    sum(x_40[tau_40, t, l] for tau_40 in 1:T_40, t in TR_ON[p]) +
    sum(s_20[i, tau_20, t, l] for i in 1:N, tau_20 in 1:T_20, t in TR_ON[p]) + 2 *
    sum(s_40[i, tau_40, t, l] for i in 1:N, tau_40 in 1:T_40, t in TR_ON[p])
    <= C_R[l]
    )

# Capacity constraint of 40' (3) (tror det er redundant at laven en for 20' også, da denne falder ind i (1))
@constraint(StochasticModel_Simple, [p = 1:P, l = 1:L],
    sum(x_40[tau_40, t, l] for tau_40 in 1:T_40, t in TR_ON[p]) +
    sum(s_40[i, tau_40, t, l] for i in 1:N, tau_40 in 1:T_40, t in TR_ON[p])
    <= C_40[l]
    )

# Loadlist constraint (4)
@constraint(StochasticModel_Simple, [i = 1:N, tau_20 = 1:T_20, t = 1:TR], 
    sum(s_20[i, tau_20, t, l] for l in 1:L) 
    <= omega[transport_keys[t]][tau_20]          # Relaxed to not enclude all containers
    )
@constraint(StochasticModel_Simple, [i = 1:N, tau_40 = 1:T_40, t = 1:TR], 
    sum(s_40[i, tau_40, t, l] for l in 1:L) 
    <= omega[transport_keys[t]][T_20 + tau_40]   # Relaxed to not enclude all containers
    )

    # Minimise unloaded containers constraint
@constraint(StochasticModel_Simple, [i = 1:N, tau_20 = 1:T_20, t = 1:TR],
    omega[transport_keys[t]][tau_20] -
    sum(s_20[i, tau_20, t, l] for l in 1:L)
    == z_20[tau_20, t]
    )
@constraint(StochasticModel_Simple, [i = 1:N, tau_40 = 1:T_40, t = 1:TR],
    omega[transport_keys[t]][T_20 + tau_40] -
    sum(s_40[i, tau_40, t, l] for l in 1:L)
    == z_40[tau_40, t]
    )

# Lower containers unload later constraint (8)
@constraint(StochasticModel_Simple, [i = 1:N, p = 1:P, l in L_O],
    sum(x_20[tau_20, t, n] + s_20[i, tau_20, t, n] for t in TR_A[p], tau_20 in 1:T_20, n in L_U if n > 0) + 
    sum(x_40[tau_40, t, n] + s_40[i, tau_40, t, n] for t in TR_A[p], tau_40 in 1:T_40, n in L_U if n > 0)
    <= M * delta[i,p,l]
    )

# Counting the overstowage (9)
@constraint(StochasticModel_Simple, [i = 1:N, p = 1:P, l in L_O], 
    sum(x_20[tau_20, t, l] + s_20[i, tau_20, t, l] for tau_20 in 1:T_20, t in TR_OV[p]) +
    sum(x_40[tau_40, t, l] + s_40[i, tau_40, t, l] for tau_40 in 1:T_40, t in TR_OV[p])
    - M*(1 - delta[i,p,l])
    <= y_O[i,p,l]
    )

3-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},3,...} with index sets:
    Dimension 1, Base.OneTo(5)
    Dimension 2, Base.OneTo(7)
    Dimension 3, [1, 2, 4, 5, 7, 9, 11, 13, 15, 17  …  110, 112, 114, 116, 118, 120, 122, 124, 125, 126]
And data, a 5×7×66 Array{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}, 3}:
[:, :, 1] =
 333 delta[1,1,1] - y_O[1,1,1] ≤ 333  …  333 delta[1,7,1] - y_O[1,7,1] ≤ 333
 333 delta[2,1,1] - y_O[2,1,1] ≤ 333     333 delta[2,7,1] - y_O[2,7,1] ≤ 333
 333 delta[3,1,1] - y_O[3,1,1] ≤ 333     333 delta[3,7,1] - y_O[3,7,1] ≤ 333
 333 delta[4,1,1] - y_O[4,1,1] ≤ 333     333 delta[4,7,1] - y_O[4,7,1] ≤ 333
 333 delta[5,1,1] - y_O[5,1,1] ≤ 333     333 delta[5,7,1] - y_O[5,7,1] ≤ 333

[:, :, 2] =
 333 delta[1,1,2] - y_O[1,1,2] ≤ 333

In [ ]:
# Set a 1% optimality gap and run the model
#set_optimizer_attribute(DeterministicModel_Simple, "MIPGap", 0.05)
set_optimizer_attribute(DeterministicModel_Simple, "TimeLimit", 10*60)
optimize!(DeterministicModel_Simple)